In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from tqdm import tqdm
from itertools import chain
from datetime import datetime,timedelta

import requests
import time
import pandas as pd
import numpy as np
import re

In [262]:
end_date=datetime.today().strftime("%Y %m %d")
end = end_date
print(end)
end = list(map(int, end.split()))
print(end)
end = datetime(end[0],end[1],end[2])

2022 03 13
[2022, 3, 13]


In [231]:
# 뉴스 url 크롤링

def news_urls_crawling(code='005930',start_date='2022 03 12', end_date=datetime.today().strftime("%Y %m %d")):
    end = end_date # '2022 03 12'
    end = list(map(int, end.split()))
    end = datetime(end[0],end[1],end[2])
    
    end_date = end.strftime("%Y.%m.%d") 

    start = start_date
    start = list(map(int, start.split()))
    start = datetime(start[0],start[1],start[2])
    start_date = start.strftime("%Y.%m.%d") 

    news_dates = [(end-timedelta(days=i)).strftime("%Y.%m.%d")   for i in range(0,(end-start).days+1)]

    driver = webdriver.Chrome('chromedriver.exe')

    url = f'https://finance.daum.net/quotes/A{code}#news/stock'

    driver.get(url)
    time.sleep(1)
    
    company = driver.find_element_by_xpath('//*[@id="favorite"]').text[1:-6]
    print(company)

    news_urls = []
    while True:
        try:
            x = driver.find_elements_by_xpath('//*[@id="boxContents"]/div[5]/div[1]/div[2]/div/ul/li')
            for i in x:
                news_date = i.find_element_by_xpath('span/p').text.split()[-1]
                if news_date in news_dates:
                    news_urls.append(i.find_element_by_xpath('span/a').get_attribute('href'))
                elif news_date not in news_dates:
                    driver.quit()
            now_page = driver.find_element_by_xpath('//*[@id="boxContents"]/div[5]/div[1]/div[2]/div/div/strong').text
            if int(now_page) % 10 == 0 and now_page != 1:
                driver.find_element_by_class_name('btnNext').click()
                time.sleep(1.5)
            else:
                [i.click()  for i in driver.find_elements_by_class_name('btnMove') if str(int(now_page)+1) == i.text]
                time.sleep(1.5)

        except Exception:
            print(f'뉴스 개수 : {len(news_urls)}')
            break
            
    return news_urls

In [232]:
# urls = news_urls_crawling(code='005930',start_date='2022 03 13', end_date='2022 03 13')

In [281]:
def news_crawling(code='005930',start_date='2022 03 12', end_date=datetime.today().strftime("%Y %m %d")):
    urls = news_urls_crawling(code=code,start_date=start_date, end_date=end_date)
    
    titles = []   # 기사 제목
    contents = [] # 기사 내용
    dates = []    # 기사 날짜

    driver = webdriver.Chrome('chromedriver.exe')

    for url in urls:
        driver.get(url)

        time.sleep(1)
        
        date = url.split('/')[-1][:8]
        date = datetime(int(x[:4]), int(x[4:6]), int(x[6:])).strftime("%Y.%m.%d") 
        dates.append()

        title = driver.find_element_by_xpath('//*[@id="boxHeader"]/div[1]/h5').text
        if '[' in title and ']' in title:
            title = title.replace(title[title.index('['):title.index(']')+1],'')
        else:
            pass
        titles.append(title)

        section = driver.find_element_by_id('dmcfContents')
        tag_p = section.find_elements_by_tag_name('p')
        tag_div = section.find_elements_by_tag_name('div')

        if len(tag_p) >= len(tag_div) and len(tag_p) > 1:
            document = [i.text for i in tag_p]
        elif len(tag_p) < len(tag_div):
            document = [i.text for i in tag_div]
        elif len(tag_p) == 1 and len(tag_div) == 1:
            document = [i.text for i in tag_div]

        document = [i.split('\n') for i in document]
        if type(document[0]) == list:
            document = list(chain(*document))
        else:
            pass

        c = []
        for content in document:
            if '[' in content and ']' in content:
                content = content.replace(content[content.index('['):content.index(']')+1],'')
            else:
                pass

            content = content.replace(content, '') if '@' in content else content
            content = content.replace(content, '') if '▶' in content else content
            content = content.replace(content, '') if '■' in content else content
            content = content.replace(content, '') if '◇' in content else content
            content = content.replace(content, '') if 'ⓒ' in content else content
            content = content.replace(content, '') if '(끝)' in content else content
            content = content.replace(content, '') if '사진=' in content else content
            content = content.replace(content, '') if '☞' in content else content
            content = re.sub('[━△▲◆]',"",content)
            c.append(content)

        contents.append(' '.join(c).rstrip())

    driver.quit()
    print('Done')
    
    data_dict = {'dates':dates,'titles':titles, 'contents':contents}
    df = pd.DataFrame(data_dict, columns=data_dict.keys())
    df = df.drop(index = df.query('contents == ""').index, axis=1)
    df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(f'datas/{company}_{start_date}~{end_date}.csv',index=False,encoding='utf-8')
    
    return df

In [241]:
df = news_crawling(code='005930',start_date='2022 03 11', end_date='2022 03 13')

삼성전자
뉴스 개수 : 250
Done
